In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
from pathlib import Path
from functools import partial

import ipywidgets as w
import ipyvuetify as v
import pdblp
import numpy as np
import pandas as pd
from IPython.display import display
from ipydatagrid import DataGrid
from sklearn.linear_model import LinearRegression

root_path = Path('/dev/projects/credit/backend/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.gui.utils import tables
from api.gui.payoff import Form
from api.gui.base import View, Tabs
from api.data.processing import get_current_data

In [ ]:
bq = BlpQuery(timeout=50000).start()
blp_dt_fmt = '%Y%m%d'

In [ ]:
bonds_fields = [
    {
        "field": "PX_LAST",
        "name": "last_price",
    },
    {
        "field": "Z_SPRD_MID",
        "name": "z_spread_mid",
    }
]

cds_fields = [
    {
        "field": "PX_LAST",
        "name": "cds_spread",
    }
]

options = {
    'calendarCodeOverride': '5D',
    "nonTradingDayFillOption": "ALL_CALENDAR_DAYS",
    "nonTradingDayFillMethod": "PREVIOUS_VALUE"
}

In [ ]:
def get_bond_hist(bond_tf, start_dp, end_dp):
    bonds_hist = bq.bdh(
        securities=[bond_tf.v_model],
        fields=[field['field'] for field in bonds_fields],
        start_date=f"{pd.to_datetime(start_dp.dp.v_model):{blp_dt_fmt}}",
        end_date=f"{pd.to_datetime(end_dp.dp.v_model):{blp_dt_fmt}}",
        options=options
    )
    bonds_hist = bonds_hist.rename({field['field']: field['name'] for field in bonds_fields}, axis=1)
    return bonds_hist

In [ ]:
def get_cds_hist(cds_tf, start_dp, end_dp):

    cds_hist = bq.bdh(
        securities=[cds_tf.v_model],
        fields=[field['field'] for field in cds_fields],
        start_date=f"{pd.to_datetime(start_dp.dp.v_model):{blp_dt_fmt}}",
        end_date=f"{pd.to_datetime(end_dp.dp.v_model):{blp_dt_fmt}}",
        options=options
    )
    cds_hist = cds_hist.rename({field['field']: field['name'] for field in cds_fields}, axis=1)
    return cds_hist

In [ ]:
def get_cds_implied_price(cds_tf, cds_hist):
    cds_implied_prices = bq.bdph(
        securities=[cds_tf.v_model],
        fields=["CDS_QUOTED_PRICE"],
        id_override=cds_hist['date'].dt.strftime(blp_dt_fmt).to_list(),
        id_override_field="SW_CURVE_DT",
        CDS_FLAT_SPREAD=cds_hist['cds_spread'].to_list(),
        id_date=True
    )
    cds_implied_prices = cds_implied_prices.rename({'SW_CURVE_DT': 'date', 'CDS_QUOTED_PRICE': 'cds_implied_price'}, axis=1)
    return cds_implied_prices

In [ ]:
def get_data_hist(bonds_hist, cds_hist, cds_implied_prices, spreads):
    data_hist = pd.concat(
        [
            bonds_hist.drop('security', axis=1).set_index('date'),
            cds_hist.drop('security', axis=1).set_index('date'),
            cds_implied_prices.drop('security', axis=1).set_index('date')
        ],
        axis=1,
    )
    
    data_hist['adj_price'] = get_adjusted_price(data_hist)
    data_hist[['cds_spread_chg', 'z_spread_chg', 'adj_price_chg']] = get_chg(data_hist, cols=['cds_spread', 'z_spread_mid', 'adj_price'])
    data_hist['price_diff'] = get_diff(data_hist, cols=['cds_implied_price', 'last_price'])
    data_hist['curve'] = get_diff(data_hist, cols=['z_spread_mid', 'cds_spread'])

    rols = RollingOLS(window).fit(data_hist[['cds_spread_chg', 'z_spread_chg']])
    data_hist['beta'] = rols.get_coef()
    data_hist['z_spread_avg'] = data_hist['z_spread_mid'].dropna().rolling(window).mean()
    data_hist['z_spread_bucket'] = get_buckets(
        data_hist['z_spread_avg'],
        bins=spreads[:-1],
        include_inf=True,
        labels=spreads
    )
    data_hist['vol'] = data_hist['adj_price_chg'].dropna().rolling(window).std().dropna() * (252 ** 0.5)
    data_hist['vol_rank'] = data_hist['vol'].rank(pct=True)
    
    return data_hist

In [ ]:
def on_click_get_history(
    widget, 
    event, 
    payload, 
    bond_tf, 
    start_dp, 
    end_dp, 
    cds_tf,
    store
):
    widget.loading = True
    try:
        bonds_hist = get_bond_hist(bond_tf, start_dp, end_dp)
        cds_hist = get_cds_hist(cds_tf, start_dp, end_dp)
        cds_implied_prices = get_cds_implied_price(cds_tf, cds_hist)
        data_hist = get_data_hist(bonds_hist, cds_hist, cds_implied_prices, spreads=store.tabs['scenarios'].to_pandas()['spread'].to_list())

        store.data.update(
            {
                'bonds_hist': bonds_hist,
                'cds_hist': cds_hist,
                'cds_implied_prices': cds_implied_prices,
                'data_hist': data_hist
            }
        )

        dg = store.tabs['history'].make_grid(data_hist)
        display(dg)
    except Exception as e:
        raise(e)
    finally:
        widget.loading = False

### Market

In [ ]:
def get_bond_kt(bond_tf):
    bonds_mkt = bq.bdp(
        securities=[bond_tf.v_model],
        fields=[field['field'] for field in bonds_fields],
    )
    bonds_mkt = bonds_mkt.rename({field['field']: field['name'] for field in bonds_fields}, axis=1)
    return bonds_mkt

In [ ]:
def get_cds_mkt(cds_tf):
    cds_mkt = bq.bdp(
        securities=[cds_tf.v_model],
        fields=[field['field'] for field in cds_fields],
    )
    cds_mkt = cds_mkt.rename({field['field']: field['name'] for field in cds_fields}, axis=1)
    return cds_mkt

In [ ]:
def get_cds_mkt_implied_price(cds_tf, spread):
    cds_mkt_implied_price = bq.bdp(
        securities=[cds_tf.v_model],
        fields=["UPFRONT_LAST"],
        overrides=[("CDS_FLAT_SPREAD", spread)]
    )
    cds_mkt_implied_price = cds_mkt_implied_price.rename({'UPFRONT_LAST': 'cds_implied_price'}, axis=1)
    return cds_mkt_implied_price

In [ ]:
def get_data_mkt(
    bonds_mkt, 
    cds_mkt, 
    cds_mkt_implied_price,
    bond_price_tf,
    bond_notional_tf,
    cds_price_tf,
    cds_notional_tf
):
    data_mkt = pd.concat(
        [
            bonds_mkt.drop('security', axis=1),
            cds_mkt.drop('security', axis=1),
            -cds_mkt_implied_price['cds_implied_price'] + 100
        ],
        axis=1,
    )

    data_mkt['bond_pnl'] = get_bond_pnl(data_mkt['last_price'], float(bond_price_tf.v_model), float(bond_notional_tf.v_model))
    data_mkt['cds_pnl'] = get_cds_pnl(data_mkt['cds_implied_price'], float(cds_price_tf.v_model), float(cds_notional_tf.v_model))
    data_mkt['total_pnl'] = data_mkt['bond_pnl'] + data_mkt['cds_pnl']
    return data_mkt

### PNL

In [ ]:
def get_yas_bond_price(bond_tf, end_dp, spreads):
    pnl_data = bq.bdph(
        securities=[bond_tf.v_model],
        fields=["YAS_BOND_PX"],
        id_override=spreads,
        id_override_field="YAS_ZSPREAD",
        overrides=[("USER_LOCAL_TRADE_DATE", f"{pd.to_datetime(end_dp.dp.v_model):{blp_dt_fmt}}")],
    )
    pnl_data = pnl_data.rename({"YAS_ZSPREAD": 'spread', "YAS_BOND_PX": 'yas_price'}, axis=1)
    return pnl_data

In [ ]:
def get_cds_beta_implied_price(cds_tf, cds_beta_implied_spread):
    cds_pnl_implied_price = bq.bdph(
        securities=[cds_tf.v_model],
        fields=["UPFRONT_LAST"],
        id_override=cds_beta_implied_spread,
        id_override_field="CDS_FLAT_SPREAD"
    )
    return -cds_pnl_implied_price['UPFRONT_LAST'] + 100

In [ ]:
def get_pnl_data(
    bond_tf,
    end_dp,
    bond_price_tf,
    bond_notional_tf,
    cds_tf,
    cds_price_tf,
    cds_notional_tf,
    changed_params,
    store
):
    pnl_params_df = store.tabs['scenarios'].to_pandas()
    data_mkt = store.data['data_mkt']

    
    if any(i in changed_params for i in ['bond_tf', 'end_dp']) or 'yas_bond_price' not in store.data:
        yas_bond_price = get_yas_bond_price(bond_tf, end_dp, spreads=pnl_params_df['spread'].to_list())  # blp
        store.data['yas_bond_price'] = yas_bond_price
    else:
        yas_bond_price = store.data['yas_bond_price']
    
    pnl_data = pnl_params_df.merge(yas_bond_price.drop('security', axis=1), on='spread')
    
    pnl_data['bond_pnl'] = get_bond_pnl(pnl_data['yas_price'], bond_price_tf.v_model, bond_notional_tf.v_model)
    pnl_data['cds_beta_implied_spread'] = get_cds_spread_from_ols(
        z_spreads=pnl_data['spread'],
        betas=pnl_data['beta'],
        current_z_spread=data_mkt['z_spread_mid'].squeeze(), 
        current_cds_spread=data_mkt['cds_spread'].squeeze()
    )
    
    if any(i in changed_params for i in ['cds_tf', 'end_dp']) or 'cds_beta_implied_price' not in store.data:
        cds_beta_implied_price = get_cds_beta_implied_price(cds_tf, cds_beta_implied_spread=pnl_data['cds_beta_implied_spread'].to_list())  # blp
        store.data['cds_beta_implied_price'] = cds_beta_implied_price
    else:
        cds_beta_implied_price = store.data['cds_beta_implied_price']
    pnl_data['cds_beta_implied_price'] = cds_beta_implied_price
    
    pnl_data['cds_pnl'] = get_cds_pnl(pnl_data['cds_beta_implied_price'], cds_price_tf.v_model, cds_notional_tf.v_model)
    pnl_data['total_pnl'] = pnl_data['bond_pnl'] + pnl_data['cds_pnl']
    
    store.data['pnl_data'] = pnl_data
    
    return pnl_data

In [ ]:
def on_change_get_data_mkt(
    widget, 
    event, 
    payload, 
    bond_tf,
    end_dp,
    cds_tf,
    bond_price_tf,
    bond_notional_tf,
    cds_spread_tf,
    cds_price_tf,
    cds_notional_tf,
    store,
    refresh=False
):
    widget.loading = True
    try:
        
        current_params = store.tabs['settings'].get_params()
        last_params = store.data.get('last_params', current_params)
        store.data['last_params'] = current_params
        changed_params = {k: v for k, v in current_params.items() if last_params[k] != v}
        print(changed_params)
        
        # for wid in [
        #     bond_tf,
        #     cds_tf,
        #     bond_price_tf,
        #     bond_notional_tf,
        #     cds_spread_tf,
        #     cds_notional_tf
        # ]:
        #     if not wid.v_model:
        #         widget.loading = False
        #         return

        if 'cds_spread_tf' in changed_params or not cds_price_tf.v_model:
            update_cds_price(cds_spread_tf=cds_spread_tf, cds_price_tf=cds_price_tf, cds_tf=cds_tf)
        
        if not 'data_mkt' in store.data and  any([i in current_params for i in ['bond_tf', 'cds_tf']]):
            bonds_mkt = get_bond_kt(bond_tf)
            cds_mkt = get_cds_mkt(cds_tf)
            cds_mkt_implied_price = get_cds_mkt_implied_price(cds_tf, spread=cds_mkt['cds_spread'].squeeze())
        else:
            bonds_mkt = store.data['bonds_mkt']
            cds_mkt = store.data['cds_mkt']
            cds_mkt_implied_price = store.data['cds_mkt_implied_price']


        data_mkt = get_data_mkt(
            bonds_mkt, 
            cds_mkt, 
            cds_mkt_implied_price,
            bond_price_tf,
            bond_notional_tf,
            cds_price_tf,
            cds_notional_tf
        )
        store.data.update(
            {
                'bonds_mkt': bonds_mkt,
                'cds_mkt': cds_mkt,
                'cds_mkt_implied_price': cds_mkt_implied_price,
                'data_mkt': data_mkt
            }
        )
        
        
        get_pnl_data(
            bond_tf,
            end_dp,
            bond_price_tf,
            bond_notional_tf,
            cds_tf,
            cds_price_tf,
            cds_notional_tf,
            changed_params,
            store
        )
    except Exception as e:
        raise(e)
    finally:
        widget.loading = False
            

In [ ]:
class RollingOLS:
    def __init__(self, window=3, **kwargs):
        self.window = window
        self.kwargs = kwargs

    def fit(self, data, window=None, y=None, X=None):
        if window:
            self.window = window
        window = self.window
        
        data = data.dropna()
        yr = data.iloc[:, 0] if not y else data[y]
        y = yr.name
        Xr = data.drop(y, axis=1) if not X else data[X]

        lms = {}
        for i, date in enumerate(data.index[window - 1:]):
            Xw = Xr[i: i + window]
            yw = yr[i: i + window]
            lm = LinearRegression(**self.kwargs).fit(Xw, yw)
            lms[date] = lm
        self.X = Xr
        self.y = yr
        self.lms = lms
        return self
    
    def get_coef(self, coef='beta'):
        if coef == 'beta':
            return pd.DataFrame({k: v.coef_ for k, v in self.lms.items()}).T.squeeze()
        else:
            return pd.Series({k: v.intercept_ for k, v in self.lms.items()})
        

In [ ]:
class DatePicker:
    dt_fmt = '%Y-%m-%d'
    def __init__(self, value=None, label=None, style=None, dense=True):
        
        if isinstance(value, pd.Timestamp):
            value = f"{value:{self.dt_fmt}}"
        
        self.make_vue_widgets(value=value, label=label, style=style, dense=dense)
    
    def make_vue_widgets(self, **kwargs):
        self.dp = v.DatePicker(
            v_model=kwargs['value'],
        )
        self.tf = v.TextField(
            v_model=None,
            label=kwargs['label'],
            clearable=True,
            dense=kwargs['dense'],
            prepend_icon="mdi-calendar",
            #readonly=True,
            v_on='menu.on',
            v_bind='menu.attrs',
            style_=kwargs['style']
        )
        w.jslink((self.dp, 'v_model'), (self.tf, 'v_model'))
        
        self.menu = v.Menu(
            v_model=None,
            close_on_content_click=False,
            nudge_right=40,
            transition="scale-transition",
            offset_y=True,
            min_width="auto",
            v_slots=[
                {
                    'name': 'activator',
                    'variable': 'menu',
                    'children': self.tf
                }
            ],
            children=[
                self.dp
            ]
        )
        
        def close_menu(*args, **kwargs):
            self.menu.v_model = False
    
        self.dp.on_event('input', close_menu)
    
    @property
    def components(self):
        return {
            'dp': self.dp,
            'tf': self.tf,
            'menu': self.menu
        }
    
    @property
    def widget(self):
        return self.menu
    
    def _ipython_display_(self):
        display(self.widget)

In [ ]:
def get_buckets(
    series, 
    bins, 
    labels=None,
    include_zero=True,
    include_inf=True
):
    
    if include_zero:
        bins = [0, *bins]
    if include_inf:
        bins = [*bins, np.inf]
    if labels is None:
        labels = pd.Series([f"{bins[i]}{f'-{bins[i+1]}' if bins[i+1] != np.inf else '+'}" for i in range(len(bins) - 2 + 1)])
    if not isinstance(labels, pd.Series):
        labels = pd.Series(labels)
    labels[len(labels)] = np.nan
    
    idx = np.digitize(series, bins) - 1
    buckets = labels.iloc[idx].set_axis(series.index)

    return buckets


def get_adjusted_price(data):
    return 1 + (100 - data['cds_implied_price']) / data['last_price']


def get_chg(data, cols):
    return data[cols].diff()


def get_diff(data, cols):
    return data[cols[0]] - data[cols[1]]


def get_bond_pnl(scen_prices, input_price, notional, scale=100):
    return (scen_prices - input_price) / scale * notional


def get_cds_pnl(cds_prices, input_price, notional, scale=100):
    return (input_price - cds_prices) / scale * notional


def get_cds_spread_from_ols(z_spreads, betas, current_z_spread, current_cds_spread):
    return (z_spreads - current_z_spread) * betas + current_cds_spread

In [ ]:
def update_cds_price(cds_spread_tf, cds_price_tf, cds_tf):
    res = bq.bdp(
        securities=[cds_tf.v_model],
        fields=["UPFRONT_LAST"],
        overrides=[("CDS_FLAT_SPREAD", float(cds_spread_tf.v_model))]
    )
    cds_price_tf.v_model = - res['UPFRONT_LAST'].squeeze() + 100

In [ ]:
# spreads = np.arange(200, 1050, 50)
pnl_params = {
    'spread' :[
        250,
        300,
        350,
        400,
        450,
        500,
        550,
        600,
        650,
        700,
        750,
        800,
        850,
        900,
        950,
        1000
    ],
    'beta': [
        0.55,
        0.55,
        0.55,
        0.55,
        0.75,
        1,   
        1.2, 
        1.4, 
        1.6, 
        1.8, 
        2,   
        2.2, 
        2.4, 
        2.6, 
        2.6, 
        2.6, 
    ]
}
params_df = pd.DataFrame(pnl_params)

In [ ]:
class ScenarioParams(View):
    def make_widgets(self):
        self.beta_spreads_ta = v.Textarea(
            v_model=params_df.to_csv(header=False, index=False),
            clearable=True,
            auto_grow=True,
            outlined=True
        )
    
    def make_view(self):
        beta_spreads_ta = self.beta_spreads_ta
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[beta_spreads_ta],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
            ]
        )
        
        self.view = param_box
    
    def to_pandas(self):
        from io import StringIO
        return pd.read_csv(StringIO(self.beta_spreads_ta.v_model), header=None).set_axis(['spread', 'beta'], axis=1)
    

In [ ]:
class CDSStrategy(View):
    def make_widgets(self):
        bond_tf = v.TextField(
            v_model=None,
            dense=True,
            label='bond',
            clearable=True
        )

        cds_tf = v.TextField(
            v_model=None,
            dense=True,
            label='cds',
            clearable=True
        )

        dl_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-download'])]
        )
        
        dlh_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-download'])]
        )
        
        refresh_btn = v.Btn(
            fab=True,
            # dark=True,
            small=True,
            plain=True,
            class_='mx-0',
            children=[v.Icon(children=['mdi-sync'])]
        )

        start_dp = DatePicker(label='start date')
        end_dp = DatePicker(label='end date', value=pd.Timestamp.today())

        bond_price_tf = v.TextField(
            v_model=None,
            label='bond price',
            clearable=True,
            dense=True,
            type="number",
            outlined=True
        )

        cds_spread_tf = v.TextField(
            v_model=None,
            label='cds spread',
            clearable=True,
            dense=True,
            type="number",
            outlined=True,
        )

        cds_price_tf = v.TextField(
            v_model=None,
            label='cds price',
            clearable=True,
            dense=True,
            type="number",
            outlined=True,
            disabled=True
        )

        bond_notional_tf = v.TextField(
            v_model=None,
            label='bond notional',
            clearable=True,
            dense=True,
            type="number",
            outlined=True
        )

        cds_notional_tf = v.TextField(
            v_model=None,
            label='cds notional',
            clearable=True,
            dense=True,
            type="number",
            outlined=True,
        )

        out = w.Output()

        self.bond_tf = bond_tf
        self.cds_tf = cds_tf
        self.dl_btn = dl_btn
        self.dlh_btn = dlh_btn
        self.refresh_btn = refresh_btn
        self.start_dp = start_dp
        self.end_dp = end_dp
        self.bond_price_tf = bond_price_tf
        self.cds_spread_tf = cds_spread_tf
        self.cds_price_tf = cds_price_tf
        self.bond_notional_tf = bond_notional_tf
        self.cds_notional_tf = cds_notional_tf
        self.out = out

    def make_view(self):
        bond_tf = self.bond_tf
        cds_tf = self.cds_tf
        dl_btn = self.dl_btn
        dlh_btn = self.dlh_btn
        refresh_btn = self.refresh_btn
        start_dp = self.start_dp
        end_dp = self.end_dp
        bond_price_tf = self.bond_price_tf
        cds_spread_tf = self.cds_spread_tf
        cds_price_tf = self.cds_price_tf
        bond_notional_tf = self.bond_notional_tf
        cds_notional_tf = self.cds_notional_tf              
        out = self.out
        
        param_box = v.Container(
            children=[
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[bond_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[cds_tf],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[start_dp.menu],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[end_dp.menu],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[dlh_btn],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[bond_price_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[cds_spread_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[cds_price_tf],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                ),
                v.Row(
                    children=[
                        v.Col(
                            cols=2,
                            children=[bond_notional_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[cds_notional_tf],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[dl_btn],
                            class_="my-0 py-0"
                        ),
                        v.Col(
                            cols=2,
                            children=[refresh_btn],
                            class_="my-0 py-0"
                        ),
                    ],
                    align_content='center',
                )
            ]
        )
        view = w.VBox(
            children=[
                param_box,
                out,
            ]
        )

        self.param_box = param_box
        self.view = view
    
    def link(self):
        bond_tf = self.bond_tf
        cds_tf = self.cds_tf
        dl_btn = self.dl_btn
        dlh_btn = self.dlh_btn
        refresh_btn = self.refresh_btn
        start_dp = self.start_dp
        end_dp = self.end_dp
        bond_price_tf = self.bond_price_tf
        cds_spread_tf = self.cds_spread_tf
        cds_price_tf = self.cds_price_tf
        bond_notional_tf = self.bond_notional_tf
        cds_notional_tf = self.cds_notional_tf              
        out = self.store.tabs['history'].out if 'history' in self.store.tabs else self.out
        
        
        # cds_spread_tf.on_event(
        #     'change',
        #     partial(
        #         update_cds_price,
        #         cds_price_tf=cds_price_tf,
        #         cds_tf=cds_tf
        #     )
        # )
        
        # cds_spread_tf.on_event(
        #     'change',
        #     partial(
        #         on_change_get_data_mkt,
        #         bond_tf=bond_tf,
        #         cds_tf=cds_tf,
        #         bond_price_tf=bond_price_tf,
        #         bond_notional_tf=bond_notional_tf,
        #         cds_spread_tf=cds_spread_tf,
        #         cds_price_tf=cds_price_tf,
        #         cds_notional_tf=cds_notional_tf,
        #         store=self.store
        #     )
        # )
        
        dl_btn.on_event(
            'click',
            partial(
                on_change_get_data_mkt,
                bond_tf=bond_tf,
                cds_tf=cds_tf,
                end_dp=end_dp,
                bond_price_tf=bond_price_tf,
                bond_notional_tf=bond_notional_tf,
                cds_spread_tf=cds_spread_tf,
                cds_price_tf=cds_price_tf,
                cds_notional_tf=cds_notional_tf,
                store=self.store
            )
        )
        
        
        dlh_btn.on_event(
            'click',
            out.capture(clear_output=True)(
                partial(
                    on_click_get_history,
                    bond_tf=bond_tf, 
                    start_dp=start_dp, 
                    end_dp=end_dp, 
                    cds_tf=cds_tf, 
                    store=self.store,
                    refresh=False
                )
            )
        )
        
        refresh_btn.on_event(
            'click',
            out.capture(clear_output=True)(
                partial(
                    on_click_get_history,
                    bond_tf=bond_tf, 
                    start_dp=start_dp, 
                    end_dp=end_dp, 
                    cds_tf=cds_tf, 
                    store=self.store,
                    refresh=True
                )
            )
        )
        
    def get_params(self):
        params = {
            'bond_tf': self.bond_tf.v_model,
            'cds_tf': self.cds_tf.v_model,
            'start_dp': self.start_dp.dp.v_model,
            'end_dp': self.end_dp.dp.v_model,
            'bond_price_tf': self.bond_price_tf.v_model,
            'cds_spread_tf': self.cds_spread_tf.v_model,
            # 'cds_price_tf': self.cds_price_tf.v_model,
            'bond_notional_tf': self.bond_notional_tf.v_model,
            'cds_notional_tf': self.cds_notional_tf.v_model,
        }
        return params

In [ ]:
class HistoryTable(View):
    def make_widgets(self):
        self.out = w.Output()
    
    def make_view(self):
        self.view = self.out
    
    def make_grid(self, data):
        dg = DataGrid(data)
        dg.auto_fit_columns = True
        self.dg = dg 
        return dg

In [ ]:
app = Tabs(
    {
        'settings': CDSStrategy(build=False),
        'scenarios': ScenarioParams(build=False),
        'history': HistoryTable(build=False)
    }
)
self = app.tabs['settings']
app

In [ ]:
self.bond_tf.v_model = 'BN492555@BGN Corp'
self.cds_tf.v_model = 'CCOL1U5 MSG1 Curncy'
self.start_dp.dp.v_model = '2021-02-26'
self.bond_price_tf.v_model = 58.20
self.cds_spread_tf.v_model = 263
# self.cds_spread_tf.fire_event('change', None)
self.bond_notional_tf.v_model = 80000000
self.cds_notional_tf.v_model = 100000000
window = 22
threshold = 0.043

In [ ]:
app.data['data_mkt']

In [ ]:
app.data['pnl_data']

In [3]:
from api.gui.cds_basis import CDSStrategy, ScenarioParams, HistoryTable

In [4]:
app = Tabs(
    {
        'settings': CDSStrategy(build=False),
        'scenarios': ScenarioParams(build=False),
        'history': HistoryTable(build=False)
    }
)
self = app.tabs['settings']

In [5]:
app

Card(children=[Tabs(children=[Tab(children=['settings']), Tab(children=['scenarios']), Tab(children=['history'…

In [6]:
self.bond_tf.v_model = 'BN492555@BGN Corp'
self.cds_tf.v_model = 'CCOL1U5 MSG1 Curncy'
self.start_dp.dp.v_model = '2021-02-26'
self.bond_price_tf.v_model = 58.20
self.cds_spread_tf.v_model = 262
# self.cds_spread_tf.fire_event('change', None)
self.bond_notional_tf.v_model = 80000000
self.cds_notional_tf.v_model = 100000000
window = 22
threshold = 0.043

In [7]:
dgm = DataGrid(app.data['data_mkt'], layout={"height":"80px", "align-content":"center"})
dgm.auto_fit_columns = True

In [8]:
dg_pnl = DataGrid(app.data['pnl_data'])
dg_pnl.auto_fit_columns = True

In [9]:
w.VBox([dgm, dg_pnl])